# Data preparation

This is a file that prepare data for creating feature store
1. A few cols were removed from the table in gold layer
2. the target col that hs True and False as values were changed to bianry 1,0
3. usre_id is kept for the feature store

In [0]:
%run ./setup

## Read data from gold layer to memory

In [0]:
# Read our churn_features table
churn_dataset = spark.table("churn_features")
display(churn_dataset)

## Convert it to pandas on spark and Remove a few unrelevant cols

In [0]:
# Convert to pandas on spark
dataset = churn_dataset.pandas_api()
dataset.describe()  
# Drop columns we don't want to use in our model
dataset = dataset.drop(columns=['address', 'email', 'firstname', 'lastname', 'creation_date', 'last_activity_date', 'last_event'])
# Drop missing values
dataset = dataset.dropna()
# print the ten first rows
dataset[:10]

In [0]:
dataset.info()

In [0]:
dataset.churn.unique()

In [0]:
dataset.loc[dataset['churn'] == True,'Churn1'] = 1
dataset.loc[dataset.churn == False,'Churn1'] = 0

In [0]:
dataset[:10]

In [0]:
dataset=dataset.drop(columns=['churn'])
dataset=dataset.rename(columns={'Churn1':'churn'})
dataset.info()

## Write to feature store

In 'Features', there should be a feature table. And a physical table 'churn_user_features' is added to catalog.schema   

In [0]:
from databricks.feature_store import FeatureStoreClient

fs = FeatureStoreClient()

try:
  #drop table if exists
  fs.drop_table('churn_user_features')
except: pass

#Note: You might need to delete the FS table using the UI
churn_feature_table = fs.create_table(
  name='churn_user_features',
  primary_keys='user_id',
  schema=dataset.spark.schema(),
  description='These features are derived from the churn_bronze_customers table in the lakehouse.  We created dummy variables for the categorical columns, cleaned up their names, and added a boolean flag for whether the customer churned or not.  No aggregations were performed.'
)

fs.write_table(df=dataset.to_spark(), name='churn_user_features', mode='overwrite')
features = fs.read_table('churn_user_features')
display(features)